<a href="https://colab.research.google.com/github/leadeeerq/machine-learning/blob/main/2_ML34_PCA_Wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

np.set_printoptions(precision=4, suppress=True, edgeitems=5, linewidth=200)

#Załadowanie danych

In [2]:
df_raw = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
df = df_raw.copy()
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [3]:
data = df.iloc[:, 1:]
target = df.iloc[:, 0]
data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
target.value_counts()

2    71
1    59
3    48
Name: 0, dtype: int64

#Podział na zbiór treningowy i testowy

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target)

print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')

X_train shape: (133, 13)
X_test shape: (45, 13)


#Standaryzacja

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)
X_train_std[:5]

array([[-0.1357,  2.0661,  0.4213,  0.6012, -0.932 , -0.8977, -1.3297,  0.8768, -1.2578,  1.1442, -1.7958, -1.0432, -0.3538],
       [ 0.413 ,  0.8186,  0.0419,  0.6012, -0.4796, -0.5347, -1.2223,  0.7152, -0.5536,  1.4864, -1.7515, -1.372 , -0.2743],
       [-0.7453, -1.2757, -3.828 , -2.7202, -0.7812, -0.4557, -1.4078, -0.6587, -2.0524, -1.3886,  0.4655, -1.098 , -0.6877],
       [-0.4649, -0.9115, -1.7792, -0.313 , -0.7812, -1.2923, -0.6365, -0.5779, -0.373 , -1.1664,  0.4212,  0.2032, -0.5541],
       [ 0.6325, -0.4835,  0.9146,  0.1441, -0.178 ,  0.4124,  0.584 , -0.6587,  0.1868, -0.6909,  0.7759,  1.6414,  0.3459]])

#PCA

In [8]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)
X_train_pca.shape

(133, 3)

In [9]:
results = pd.DataFrame(data={'explained_variance_ratio': pca.explained_variance_ratio_})
results['cumulative'] = results['explained_variance_ratio'].cumsum()
results['component'] = results.index + 1
results

,explained_variance_ratio,cumulative,component
0,0.360800,0.360800,1
1,0.190729,0.551529,2
2,0.118596,0.670124,3


In [10]:
fig = go.Figure(data=[go.Bar(x=results['component'], y=results['explained_variance_ratio'], name='explained variance ratio'),
                      go.Scatter(x=results['component'], y=results['cumulative'], name='cumulative explained variance')],
                layout=go.Layout(title=f'PCA - {pca.n_components_} components', width=950, template='plotly_dark'))
fig.show()

In [11]:
X_train_pca_df = pd.DataFrame(data=np.c_[X_train_pca, y_train], columns=['pca1', 'pca2', 'pca3', 'target'])
X_train_pca_df.head()

,pca1,pca2,pca3,target
0,3.511861,-1.198578,0.117979,3.0
1,2.763289,-1.502492,-0.198074,3.0
2,0.816899,2.698233,-4.982084,2.0
3,0.429641,2.203467,-1.766523,2.0
4,-1.849832,0.286451,0.779605,1.0


In [12]:
px.scatter_3d(X_train_pca_df, x='pca1', y='pca2', z='pca3', color='target', template='plotly_dark', width=950)

In [13]:
X_train_pca[:5]

array([[ 3.5119, -1.1986,  0.118 ],
       [ 2.7633, -1.5025, -0.1981],
       [ 0.8169,  2.6982, -4.9821],
       [ 0.4296,  2.2035, -1.7665],
       [-1.8498,  0.2865,  0.7796]])

In [14]:
X_test_pca[:5]

array([[-2.5085, -1.3801,  0.6207],
       [ 3.3055, -2.1837, -0.0774],
       [-2.698 , -1.6113, -0.2711],
       [ 0.7325, -0.0042, -0.8709],
       [ 1.1681,  2.1659, -0.1363]])